## Final Project Submission

Please fill out:
* Student name: Sylvia Sarange Manono
* Student pace: Part time
* Scheduled project review date/time: 
* Instructor name: Samuel G. Mwangi
* Blog post URL:

# Microsoft Movie Venture

Author: Sylvia Sarange Manono

## Overview

The goal of this project is to perform exploratory data analysis on movie data to provide insights that can help guide Microsoft's new movie studio on what types of films to produce.

## Business Problem

With more and more movie studios opting to distribute their content on their own platforms, successful entertainment companies today are increasingly dependent on original content. Additionally, major tech companies like Netflix, Amazon, and Apple have been investing heavily in creating original video content.For instance, at its inception, Netflix, one of the pioneer movie streaming sites rose to popularity by distributing content licensed from established movie studios. However, with the growing demand for streaming services, these studios established their own streaming platforms, forcing Netflix to invest heavily in its own content.  

Microsoft has decided to follow suit by establishing a new movie studio division. However, Microsoft has little prior experience in the movie production business. To assist in defining the strategic direction for the new studio, data-driven insights are needed on current box office trends and successful movie genres, styles, and so on. This project seeks to provide actionable insights from explortatory data analysis to inform Microsoft's decision and ensure the venture is successful and profitable.

## Data

Movie datasets are obtained

Movie datasets are obtained from the following two sources:

1. im.db
2. tn.movie_budgets.csv

In [2]:
conn = sqlite3.connect('data/im.db')

NameError: name 'sqlite3' is not defined